In [1]:
import os
import ee
import geemap

In [225]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AX4XfWjI0wTUPh7Jt7zUyN1esFljd56PlCLaacXs-EtvowRJFAHhN7dDQWw


## Main machine learning models used in Earth Observation
- Supervised learning;
- Unsupervised learning.
We will only focus on supervised learning here.

## [Supervised learning](http://localhost:8888/notebooks/Day%202/geemap-master/examples/notebooks/31_unsupervised_classification.ipynb)
The steps involved in the process of image classification with machine learning include:
- Define the image to classifiy;
- Collect the training data;
- Instantiate a classifier;
- Train a classifier;
- Classify;
- Estimate classification error with independent validation data.

The classifier available for supervised machine learning in google earth engine include:
![title](https://i.imgur.com/vROsEiq.png)

### I.1. Define the image to classifiy

In [226]:
# Upload the shapefile of the ROI
sa22    = geemap.shp_to_ee('/Users/noe/Desktop/SA4/ADM4_PCODE_ZA1011001.shp')
Map.addLayer(sa22, {}, 'ROI')
Map

In [227]:
# Add landsat8 TOA to the ROI
dataset = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA").filterBounds(sa22).filterDate('2021-01-01','2021-12-31').filterMetadata('CLOUD_COVER','less_than',70)
rgbVis  = {
  'min': 0.0,
  'max': 0.3,
  'gamma':1.4,
  'bands': ['B4_median', 'B3_median', 'B2_median'],
}
# ///////////////// Use Median Band in parameters///////////////////////

median = dataset.reduce(ee.Reducer.median())

# ////////////////Mosaic these two image collections///////////////////

mosaic_1 = ee.ImageCollection.fromImages([median]).mosaic()

# ////////////////////////Clip by assets or roi////////////////////////

landsat_8_2021 = mosaic_1.clip(sa22)

# /////////////// Display the clipped ImageCollection/////////////////

Map.addLayer(landsat_8_2021, rgbVis, 'landsat_roi')
Map

### I.2. Collect the training data
The collection of training data can be done in two ways:
 - Manual collection;
 - Automatic collection.
    
The manual collection is performed in the cases where there is no data available. The automatic selection uses existing classification map, to generate the training data.

In [228]:
# Manual collection


In [229]:
# Automatic collection
# Load the classified data that will help to collect the data
### Legend and colors dictionary
dico = {"names": ["Contiguous (indigenous) Forest (combined very high, high, medium)", "Contiguous Low Forest & Thicket (combined classes)", "Dense Forest & Woodland (35 - 75% cc)", "Open Woodland (10 - 35% cc)", "Contiguous & Dense Planted Forest (combined classes)", "Open & Sparse Planted Forest", "Temporary Unplanted Forest", "Low Shrubland (other regions)", "Low Shrubland (Fynbos)", "Low Shrubland (Succulent Karoo)", "Low Shrubland (Nama Karoo)", "Sparsely Wooded Grassland (5 - 10% cc)", "Natural Grassland", "Natural Rivers", "Natural Estuaries & Lagoons", "Natural Ocean, Coastal", "Natural Lakes", "Natural Pans (flooded @ obsv time)", "Artificial Dams (incl. canals)", "Artificial Sewage Ponds", "Artificial Flooded Mine Pits", "Herbaceous Wetlands (currently mapped)", "Herbaceous Wetlands (previous mapped extent)", "Mangrove Wetlands", "Natural Rock Surfaces", "Dry Pans", "Eroded Lands", "Sand Dunes (terrestrial)", "Coastal Sand Dunes & Beach Sand", "Bare Riverbed Material", "Other Bare", "Cultivated Commercial Permanent Orchards", "Cultivated Commercial Permanent Vines", "Cultivated Commercial Sugarcane Pivot Irrigated", "Commercial Permanent Pineapples", "Cultivated Commercial Sugarcane Non-Pivot (all other)", "Cultivated Emerging Farmer Sugarcane Non-Pivot (all other)", "Commercial Annuals Pivot Irrigated", "Commercial Annuals Non-Pivot Irrigated", "Commercial Annuals Crops Rain-Fed / Dryland / Non-Irrigated", "Subsistence / Small-Scale Annual Crops", "Fallow Land & Old Fields (Trees)", "Fallow Land & Old Fields (Bush)", "Fallow Land & Old Fields (Grass)", "Fallow Land & Old Fields (Bare)", "Fallow Land & Old Fields (Low Shrub)", "Residential Formal (Tree)", "Residential Formal (Bush)", "Residential Formal (low veg / grass)", "Residential Formal (Bare)", "Residential Informal (Tree)", "Residential Informal (Bush)", "Residential Informal (low veg / grass)", "Residential Informal (Bare)", "Village Scattered (bare only)", "Village Dense (bare only)", "Smallholdings (Tree)", "Smallholdings (Bush)", "Smallholdings (low veg / grass)", "Smallholdings (Bare)", "Urban Recreational Fields (Tree)", "Urban Recreational Fields (Bush)", "Urban Recreational Fields (Grass)", "Urban Recreational Fields (Bare)", "Commercial", "Industrial", "Roads & Rail (Major Linear)", "Mines: Surface Infrastructure", "Mines: Extraction Sites: Open Cast & Quarries combined", "Mines: Extraction Sites: Salt Mines", "Mines: Waste (Tailings) & Resource Dumps", "Land-fills", "Fallow Land & Old Fields (wetlands)"], "colors": ["#F2F2F2", "#065106", "#005F00", "#008500", "#F74006", "#F9764D", "#F9906C", "#B8ABD1", "#8FAB39", "#AC92C5", "#AC9CDA", "#85D285", "#D2B485", "#00009F", "#041FA7", "#0639AB", "#0D50AC", "#125FAC", "#1373B4", "#1D81B6", "#1F8EB8", "#06DEDC", "#06E0D0", "#9F1FEC", "#ffffe0", "#DCDAC5", "#F9E0E0", "#F9F9C5", "#F9F9A7", "#CDD2E0", "#ffffe0", "#A62C39", "#B31F5C", "#DB0000", "#9F3978", "#FF0000", "#F64D6C", "#381A12", "#521F1C", "#85402C", "#C5735F", "#C1436C", "#C55E82", "#D27592", "#E0AAB8", "#DB90A9", "#ECDB0F", "#F6EC13", "#F9F81F", "#FFFF29", "#EC82EC", "#F691E0", "#F99FCF", "#FFC5CF", "#ECC500", "#FFD91F", "#AC7879", "#B89192", "#C49C9E", "#D2B8B8", "#BFFF00", "#33FF33", "#66FF66", "#99FF99", "#C49F0D", "#8F8506", "#F9DD03", "#FFFF00", "#B30606", "#C50606", "#D21D1A", "#F95479", "#6CE7DC"]}

sanlcd2020 = ee.Image('users/noecareme/SA_NLC_2020_GEO').select('b1').clip(sa22)
Map.addLayer(sanlcd2020, {'min':1, 'max':73,'palette':dico['colors']}, 'SANLC2020')
Map

In [230]:
# Make the training dataset.
points = sanlcd2020.sample(**{
    'region': sa22,
    'scale': 20,
    'numPixels': 1000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

Map.addLayer(points, {}, 'training', False)
Map

In [231]:
print(points.size().getInfo())

In [232]:
print(points.first().getInfo())

In [233]:
### Split the 1000 points into training and testing
# Use these bands for prediction.
bands = ['B1_median', 'B2_median', 'B3_median', 'B4_median', 'B5_median', 'B6_median', 'B7_median']

# This property of the table stores the land cover labels.
label = 'b1'

# Overlay the points on the imagery to get training.
sample = landsat_8_2020.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 20
})

# Adds a column of deterministic pseudorandom numbers. 
sample = sample.randomColumn()

split = 0.7

training = sample.filter(ee.Filter.lt('random', split))
validation = sample.filter(ee.Filter.gte('random', split))

In [234]:
print(training.first().getInfo())

In [235]:
validation.first().getInfo()

### Train the classifier

In this examples, we will use random forest classification.

In [236]:
classifier = ee.Classifier.smileRandomForest(5).train(training, label, bands)

### Classify the image and display the results

In [237]:
# Classify the image with the same bands used for training.
result = landsat_8_2020.select(bands).classify(classifier)

# # Display the clusters with random colors.
Map.addLayer(result, {'min':1, 'max':73,'palette':dico['colors']}, 'classified') #result.randomVisualizer()
Map

### Accuracy Assessment

In [238]:
train_accuracy = classifier.confusionMatrix()

In [239]:
import pandas as pd
D = pd.DataFrame(train_accuracy.getInfo())
D.head()

In [240]:
train_accuracy.accuracy().getInfo()

In [241]:
train_accuracy.kappa().getInfo()

In [242]:
PA = pd.DataFrame(train_accuracy.producersAccuracy().getInfo())
PA.head()

In [243]:
CA = pd.DataFrame(train_accuracy.consumersAccuracy().getInfo())
CA.head()

#### Validation dataset

In [244]:
validated = validation.classify(classifier)

In [245]:
validation_accuracy = validated.errorMatrix('b1', 'classification')

In [246]:
VA = pd.DataFrame(validation_accuracy.getInfo())
VA.head()

In [247]:
validation_accuracy.accuracy().getInfo()

In [248]:
validation_accuracy.kappa().getInfo()

In [249]:
VPA = validation_accuracy.producersAccuracy().getInfo()
pd.DataFrame(VPA).head()

In [250]:
VCA = pd.DataFrame(validation_accuracy.consumersAccuracy().getInfo())
pd.DataFrame(VCA).head()

## Features engineering
This section consists of adding more information to the image, in order to help the classifier, perform more accurate classification.

In [251]:
alos = ee.Image("JAXA/ALOS/AW3D30/V2_2")
def addIndices(image):
    ndvi = image.normalizedDifference(['B5_median', 'B4_median']).rename(['ndvi'])
    ndbi = image.normalizedDifference(['B6_median', 'B5_median']).rename(['ndbi'])
    mndwi = image.normalizedDifference(['B3_median', 'B6_median']).rename(['mndwi']) 
    bsi = image.expression(
      '(( X + Y ) - (NIR + B)) /(( X + Y ) + (NIR + B)) ', {
        'X': image.select('B6_median'), #swir1
        'Y': image.select('B4_median'),  #red
        'NIR': image.select('B5_median'), # nir
        'B': image.select('B2_median'), # blue
  }).rename('bsi')
    savi = image.expression(
  '(2 * NIR + 1 - sqrt(pow((2 * NIR + 1), 2) - 8 * (NIR - RED)) ) / 2', 
  {'NIR': image.select('B5_median'),'RED': image.select('B4_median')}).rename('savi')
    return image.addBands(ndvi).addBands(ndbi).addBands(mndwi).addBands(bsi).addBands(savi)

landsat_8_2020 = addIndices(landsat_8_2020)
elev = alos.select('AVE_DSM').divide(2000).rename('elev')
slope = ee.Terrain.slope(alos.select('AVE_DSM')).divide(30).rename('slope')
landsat_8_2020 = landsat_8_2020.addBands(elev).addBands(slope)


In [252]:
### Split the 1000 points into training and testing
# Use these bands for prediction.
bands = ['B1_median', 'B2_median', 'B3_median', 'B4_median', 'B5_median', 'B6_median', 'B7_median']

# This property of the table stores the land cover labels.
label = 'b1'

# Overlay the points on the imagery to get training.
sample = landsat_8_2020.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 20
})

# Adds a column of deterministic pseudorandom numbers. 
sample = sample.randomColumn()

split = 0.7

training = sample.filter(ee.Filter.lt('random', split))
validation = sample.filter(ee.Filter.gte('random', split))

In [253]:
classifier = ee.Classifier.smileRandomForest(10).train(training, label, bands)

In [254]:
# Classify the image with the same bands used for training.
result = landsat_8_2020.select(bands).classify(classifier)

# # Display the clusters with random colors.
Map.addLayer(result, {'min':1, 'max':73,'palette':dico['colors']}, 'classified') #result.randomVisualizer()
Map

In [255]:
train_accuracy = classifier.confusionMatrix()

In [256]:
import pandas as pd
D = pd.DataFrame(train_accuracy.getInfo())
D.head()

In [257]:
train_accuracy.accuracy().getInfo()

In [258]:
train_accuracy.kappa().getInfo()

In [259]:
validated = validation.classify(classifier)

In [260]:
validation_accuracy.accuracy().getInfo()

In [261]:
validation_accuracy = validated.errorMatrix('b1', 'classification')

In [262]:
validation_accuracy.kappa().getInfo()

### Test the model on SANLC 2018

In [263]:
# Add landsat8 TOA to the ROI
dataset = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA").filterBounds(sa22).filterDate('2018-01-01','2018-12-31').filterMetadata('CLOUD_COVER','less_than',70)
rgbVis  = {
  'min': 0.0,
  'max': 0.3,
  'gamma':1.4,
  'bands': ['B4_median', 'B3_median', 'B2_median'],
}
# ///////////////// Use Median Band in parameters///////////////////////

median = dataset.reduce(ee.Reducer.median())

# ////////////////Mosaic these two image collections///////////////////

mosaic_1 = ee.ImageCollection.fromImages([median]).mosaic()

# ////////////////////////Clip by assets or roi////////////////////////

landsat_8_2018 = mosaic_1.clip(sa22)

# /////////////// Display the clipped ImageCollection/////////////////

Map.addLayer(landsat_8_2018, rgbVis, 'landsat_roi18')
#Map

In [264]:
sanlcd2018 = ee.Image('projects/sat-io/open-datasets/landcover/SA_NLC_2018').select('b1').clip(sa22)

# Make the training dataset.
points1 = sanlcd2018.sample(**{
    'region': sa22,
    'scale': 20,
    'numPixels': 800,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

In [265]:
### Split the 1000 points into training and testing
# Use these bands for prediction.
bands = ['B1_median', 'B2_median', 'B3_median', 'B4_median', 'B5_median', 'B6_median', 'B7_median']

# This property of the table stores the land cover labels.
label = 'b1'

# Overlay the points on the imagery to get training.
sample1 = landsat_8_2018.select(bands).sampleRegions(**{
  'collection': points1,
  'properties': [label],
  'scale': 20
})

In [266]:
test18 = sample1.classify(classifier)

In [267]:
test_accuracy18 = test18.errorMatrix('b1', 'classification')

In [268]:
T18 = pd.DataFrame(test_accuracy18.getInfo())
pd.DataFrame(T18).head()

In [269]:
test_accuracy18.accuracy().getInfo()

In [270]:
test_accuracy18.kappa().getInfo()

In [271]:
TPA = test_accuracy18.producersAccuracy().getInfo()
pd.DataFrame(TPA).head()

In [272]:
TCA = pd.DataFrame(test_accuracy18.consumersAccuracy().getInfo())
pd.DataFrame(TCA).head()

## Apply the model to the image to classify 

In [273]:
# Classify the image with the same bands used for training.
result1 = landsat_8_2021.select(bands).classify(classifier)

# # Display the clusters with random colors.
Map.addLayer(result1, {'min':1, 'max':73,'palette':dico['colors']}, 'classified_2021') #result.randomVisualizer()
Map

## Hyperparameter optimization

This section consists of finding the best number of trees, that leads to the highest accuracy in the random forest.

In [274]:
#####/Classify the test FeatureCollection.#####*

validation_2020 = validation.classify(classifier)

# Tune the numberOfTrees parameter.
numTreesList = ee.List.sequence(10, 350, 10)

############ Functions to loop through the number of trees ##########/

def accuracies_2020(numTrees):
    classifier_2020 = ee.Classifier.smileRandomForest(numTrees).train(training, label, bands)
    # Here we are classifying a table instead of an image
    # Classifiers work on both images and tables
    return validation.classify(classifier_2020).errorMatrix(label, 'classification').accuracy(),sample1.classify(classifier_2020).errorMatrix(label, 'classification').accuracy()

T = [accuracies_2020(numTrees) for numTrees in numTreesList.getInfo()]
t = []
for i in range(len(T)):
    t.append(T[i][0].getInfo())
r = pd.DataFrame(t)

t1 = []
for i in range(len(T)):
    t1.append(T[i][1].getInfo())
r1 = pd.DataFrame(t1)

import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(figsize=(12, 6))

x = numTreesList.getInfo()
y = r[0]
z = r1[0]
ax.plot(x,y, color='blue',label="Validation SANLC 2020")
ax.plot(x,z, color='black',label="Test SANLC 2018")
plt.xlabel("NumTrees")
plt.ylabel("Accuracy")
plt.legend(loc="lower right")
plt.show()

## Best model

In [275]:
classifier = ee.Classifier.smileRandomForest(50).train(training, label, bands)

In [276]:
# Classify the image with the same bands used for training.
result = landsat_8_2020.select(bands).classify(classifier)

# # Display the clusters with random colors.
Map.addLayer(result, {'min':1, 'max':73,'palette':dico['colors']}, 'Best_classified_2020') #result.randomVisualizer()
Map

In [277]:
train_accuracy = classifier.confusionMatrix()

In [278]:
import pandas as pd
D = pd.DataFrame(train_accuracy.getInfo())
D.head()

In [279]:
train_accuracy.accuracy().getInfo()

In [280]:
train_accuracy.kappa().getInfo()

In [ ]:
validated = validation.classify(classifier)
validation_accuracy.accuracy().getInfo()
validation_accuracy = validated.errorMatrix('b1', 'classification')
validation_accuracy.kappa().getInfo()

In [ ]:
test18 = sample1.classify(classifier)
test_accuracy18 = test18.errorMatrix('b1', 'classification')
T18 = pd.DataFrame(test_accuracy18.getInfo())
pd.DataFrame(T18).head()
test_accuracy18.accuracy().getInfo()
test_accuracy18.kappa().getInfo()
TPA = test_accuracy18.producersAccuracy().getInfo()
pd.DataFrame(TPA).head()
TCA = pd.DataFrame(test_accuracy18.consumersAccuracy().getInfo())
pd.DataFrame(TCA).head()

In [ ]:
## Contribution of each bands

In [ ]:
print(classifier.explain())